{{title_s1_1}}

This notebook demonstrates working with Sentinel-1 RTC imagery that has been processed on the [ASF On-Demand server](https://docs.asf.alaska.edu/vertex/manual/) and downloaded locally. 

The downloaded time series of Sentinel-1 imagery is very large. We demonstrate strategies for reading data of this nature into memory by creating a *virtual copy* of the data. 

:::{important}
As mentioned, the steps shown in this notebook involve downloading and extracting large volumes of data. **It is not necessary to do this to follow the rest of the content in the tutorial**. We include the demonstration for the purposes of completeness and to help users who may be in this situation.

***To skip downloading the data and proceed with the tutorial, use the VRT files located in the `../tutorial2/data/` directory of the tutorial repository.***
:::

::::{tab-set}
:::{tab-item} Outline   
(content:section_A)=
**[A. Prepare to read data into memory](#a-prepare-to-read-data-into-memory)**  
- {{a1_s1_nb1}}  
- {{a2_s1_nb1}}  

(content:section_B)=
**[B. Read data](#b-read-data)**  
- {{b1_s1_nb1}}  
:::

:::{tab-item} Learning goals

{{concepts}}
- Understand local file storage and create virtual datasets from locally stored files
- Read larger-than-memory data into memory
- Use VRT files to create 3-d Xarray data cubes from stack of 2-d image files
:::
::::
:::{admonition} ASF Data Access
You can download the RTC-processed backscatter time series [here](https://zenodo.org/record/7236413#.Y1rNi37MJ-0). This tutorial starts from the point of having the data downloaded and unzipped in a directory. See the path to the directory location and the structure of the directory holding the unzipped files below. 
:::

Expand the next cell to see specific packages used in this notebook and relevant system and version information.

In [1]:
#%xmode minimal
import os
import pathlib
import re

import geopandas as gpd
import markdown
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pathlib
import rioxarray as rio
import xarray as xr

from typing import List

In [2]:
cwd = pathlib.Path.cwd()
tutorial2_dir = pathlib.Path(cwd).parent

{{break}}

## A. Prepare to read data into memory

### {{a1_s1_nb1}}

After the data is extracted from the compressed files, we have a directory containing sub-directories for each Sentinel-1 image acquisition (scene). Within each sub-directory are all of the files associated with that scene. For more information about the files contained in each directory, see this [section](https://hyp3-docs.asf.alaska.edu/guides/rtc_product_guide/#image-files) of the ASF Sentinel-1 RTC Product Guide.

The directory should look like the diagram below:

```
.
└── s1_asf_data
    ├── S1A_IW_20210502T121414_DVP_RTC30_G_gpuned_1424
    │   ├── S1A_IW_20220214T121353_DVP_RTC30_G_gpuned_51E7_VH.tif.xml
    │   ├── S1A_IW_20220214T121353_DVP_RTC30_G_gpuned_51E7_rgb.kmz
    │   ├── S1A_IW_20220214T121353_DVP_RTC30_G_gpuned_51E7_shape.prj
    │   ├── S1A_IW_20220214T121353_DVP_RTC30_G_gpuned_51E7.png.aux.xml
    │   ├── S1A_IW_20220214T121353_DVP_RTC30_G_gpuned_51E7.README.md.txt
    │   ├── S1A_IW_20220214T121353_DVP_RTC30_G_gpuned_51E7_rgb.png.aux.xml
    │   ├── S1A_IW_20220214T121353_DVP_RTC30_G_gpuned_51E7_rgb.png
    │   ├── S1A_IW_20220214T121353_DVP_RTC30_G_gpuned_51E7_VV.tif.xml
    │   ├── S1A_IW_20220214T121353_DVP_RTC30_G_gpuned_51E7.png
    │   ├── S1A_IW_20220214T121353_DVP_RTC30_G_gpuned_51E7_rgb.png.xml
    │   ├── S1A_IW_20220214T121353_DVP_RTC30_G_gpuned_51E7.png.xml
    │   ├── S1A_IW_20220214T121353_DVP_RTC30_G_gpuned_51E7_shape.shp
    │   ├── S1A_IW_20220214T121353_DVP_RTC30_G_gpuned_51E7_VH.tif
    │   ├── S1A_IW_20220214T121353_DVP_RTC30_G_gpuned_51E7.log
    │   ├── S1A_IW_20220214T121353_DVP_RTC30_G_gpuned_51E7_VV.tif
    │   ├── S1A_IW_20220214T121353_DVP_RTC30_G_gpuned_51E7.kmz
    │   ├── S1A_IW_20220214T121353_DVP_RTC30_G_gpuned_51E7_shape.dbf
    │   ├── S1A_IW_20220214T121353_DVP_RTC30_G_gpuned_51E7_ls_map.tif.xml
    │   ├── S1A_IW_20220214T121353_DVP_RTC30_G_gpuned_51E7_shape.shx
    │   └── S1A_IW_20220214T121353_DVP_RTC30_G_gpuned_51E7_ls_map.tif
    └── S1A_IW_20210505T000307_DVP_RTC30_G_gpuned_54B1
        └── ...
```



To build GDAL VRT files, we need to pass a list of the input datasets. This means that we need to extract the file path to every file associated with each variable (VV, VH and L-S). 

:::{note}
If you are following along on your own computer, be sure to set `path_to_rtcs` to the location of the downloaded files on your own computer.
:::

In [4]:
path_to_rtcs = "data/subset_asf_rtcs"

In [5]:
# Path to directory holding downloaded data
s1_asf_data = pathlib.Path(cwd.parent, path_to_rtcs)


Make a list of all scenes:

In [6]:
scenes_ls = os.listdir(s1_asf_data)

The below function makes a list of all file names across the individual scene directory for VV, VH, ls_map and README files.

In [7]:
def extract_fnames(data_path: str, scene_name:str, variable:str):
    # Make list of files within each scene directory in data directory
    scene_files_ls = os.listdir(os.path.join(data_path, scene_name))

    if variable in ['vv','vh']:
        scene_files = [fname for fname in scene_files_ls if fname.endswith(f"_{variable.upper()}.tif")]
    
    elif variable == 'ls_map':
        scene_files = [fname for fname in scene_files_ls if fname.endswith(f"_ls_map.tif")]
    
    elif variable == 'readme':
        scene_files = [file for file in scene_files_ls if file.endswith("README.md.txt")]
   
    return scene_files




Below is the output of `extract_fnames()` for two one scene directory. Note that `os.listdir()` **does not** preserve the order of the subdirectories as listed on disk. This is okay because we will ensure that the files are sorted in chronological order later.

In [8]:
print(extract_fnames(s1_asf_data, scenes_ls[0], 'vv'))
print(extract_fnames(s1_asf_data, scenes_ls[0], 'vh'))
print(extract_fnames(s1_asf_data, scenes_ls[0], 'ls_map'))
print(extract_fnames(s1_asf_data, scenes_ls[0], 'readme'))

['S1A_IW_20210514T121349_DVP_RTC30_G_gpuned_971C_VV.tif']
['S1A_IW_20210514T121349_DVP_RTC30_G_gpuned_971C_VH.tif']
['S1A_IW_20210514T121349_DVP_RTC30_G_gpuned_971C_ls_map.tif']
['S1A_IW_20210514T121349_DVP_RTC30_G_gpuned_971C.README.md.txt']


We need to attach the filenames to the path of each file so that we end up with a list of the full paths to the VV and VH band imagery, layover-shadow maps and README files. Within this step, we will also add checks to ensure that the process is doing what we want, which is to create lists of filepaths for each character **with the same order across lists** so that we can combine the lists into a multivariate time series. As noted above, it is okay that the lists are not in chronological order, as long as they are in **the same** order across variables. 

In [9]:
def make_filepath_lists(asf_s1_data_path: str, variable:str):
    """ For a single variable (vv, vh, ls_map or readme), make list of 
    full filepath for each file in time series. Also return dates to ensure
    extraction happens in correct order for each variable
    Return tuple with form (filepaths list, dates list)"""
    scenes_ls = os.listdir(asf_s1_data_path)

    fpaths, dates_ls = [],[]

    for element in range(len(scenes_ls)):

        # Extract filenames of each file of interest
        files_of_interest = extract_fnames(asf_s1_data_path, scenes_ls[element], variable)
        # Make full path with filename for each variable
        path = os.path.join(
            asf_s1_data_path, scenes_ls[element], files_of_interest[0]
        )
        #extract dates to make sure dates are identical across variable lists
        date = pathlib.Path(path).stem.split("_")[2]
        
        dates_ls.append(date)
        fpaths.append(path)
    
    return (fpaths, dates_ls)

    

    

In [10]:
def create_filenames_dict(rtc_path, variables_ls):

    keys, filepaths, dates = [],[],[]
    for variable in variables_ls:
        keys.append(variable)

        filespaths_list, dates_list = make_filepath_lists(rtc_path, variable)
        filepaths.append(filespaths_list)
        dates.append(dates_list)
    
    #make dict of variable names (keys) and associated filepaths
    filepaths_dict = dict(zip(keys, filepaths))

    #make sure that dates are identical across all lists
    assert all(lst == dates[0] for lst in dates) == True
    #make sure length of each variable list is the same 
    assert len(list(set([len(v) for k,v in filepaths_dict.items()]))) == 1
    


    #make dict of variable names (keys) and associated filepaths
    filepaths_dict = dict(zip(keys, filepaths))
    return filepaths_dict

In [11]:
filepaths_dict = create_filenames_dict(s1_asf_data, ['vv','vh','ls_map','readme'])

`filepaths_dict` has a key for each file type:

In [12]:
filepaths_dict.keys()

dict_keys(['vv', 'vh', 'ls_map', 'readme'])

The values are the paths to each file.

### {{a2_s1_nb1}}

We will be using the `gdalbuildvrt` command. You can find out more about it [here](https://manpages.ubuntu.com/manpages/bionic/man1/gdalbuildvrt.1.html). This command creates a *virtual* GDAL dataset given a list of other GDAL datasets (the Sentinel-1 scenes). `gdalbuildvrt` can make a VRT that either tiles the listed files into a large spatial mosaic, or places them each in a separate band of the VRT. Because we are dealing with a temporal stack of images we want to use the `-separate` flag to place each file into a band of the VRT. </br>

Here is where we use the list of file paths we created above. For each variable, write the list of file paths to a `.txt` file which is then passed to `gdalbuildvrt`. 

In [13]:
def create_vrt_object(fpaths_dict: dict, variable: str):
    """Function to create VRT files for each variable given a
    list of file paths fo that variable."""
    
    #switch 'ls' to match variable name in dict
    if variable == "ls":
        variable = "ls_map"

    # Write file paths to txt file
    fpath_input_txt = pathlib.Path(f"../data/s1_{variable}_fpaths.txt")

    # Specify location of vrt file- note that we use
    # tutorial2_dir path defined at top of notebook
    output_vrt_path = f"../data/vrt_files/s1_stack_{variable}.vrt"

    # Write file paths to txt file
    with open(fpath_input_txt, "w") as fp:
        for item in fpaths_dict[f"{variable}"]:
            #print(item)
            fp.write(f"{item}\n")

    !gdalbuildvrt -separate -input_file_list {fpath_input_txt} {output_vrt_path}

In [14]:
create_vrt_object(filepaths_dict, "vv")
create_vrt_object(filepaths_dict, "vh")
create_vrt_object(filepaths_dict, "ls")

0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.


## B. Read data

In [15]:
ds_vv = xr.open_dataset("../data/vrt_files/s1_stack_vv.vrt", chunks="auto")
ds_vh = xr.open_dataset("../data/vrt_files/s1_stack_vh.vrt", chunks="auto")
ds_ls = xr.open_dataset("../data/vrt_files/s1_stack_ls_map.vrt", chunks="auto")

In [16]:
ds_vv

<xarray.Dataset> Size: 5GB
Dimensions:      (band: 5, x: 17442, y: 13377)
Coordinates:
  * band         (band) int64 40B 1 2 3 4 5
  * x            (x) float64 140kB 3.834e+05 3.834e+05 ... 9.066e+05 9.066e+05
  * y            (y) float64 107kB 3.309e+06 3.309e+06 ... 2.907e+06 2.907e+06
    spatial_ref  int64 8B ...
Data variables:
    band_data    (band, y, x) float32 5GB dask.array<chunksize=(5, 2560, 2560), meta=np.ndarray>

Building the `VRT` object assigns every object in the .txt file to a different band. In doing this, we lose the metadata that is associated with the files. The next notebook walks through the steps of reconstructing necessary metadata stored in file names and auxiliary files and attaching it to the Xarray objects read from VRTs.

### {{b1_s1_nb1}}

```{image} ../imgs/s1_chunks.png
:alt chunks_image
:align_center
```

Each variable in `ds` has a total shape of (103, 13379, 1742) and is 89.59 GB. It is chunked so that each chunk is (11, 1536, 1536) and 99 MB, with 1080 total chunks per variable. 

Depending on your use-case, you may want to adjust the chunking of the object. For example, if you are interested in analyzing variability along the temporal dimension, it might make sense to re-chunk the dataset so that operations along the that dimension are more easily parallelized. For more detail, see the {term}`chunking` discussion in [Relevant Concepts](../../background/relevant_concepts.md) and the [Parallel Computing with Dask](https://docs.xarray.dev/en/stable/user-guide/dask.html) section of the Xarray documentation. 

## Conclusion

This notebook demonstrated reading large data into memory by creating a virtual dataset that references that full dataset without directly reading it. 

However, we also saw that reading the data in this way produces an object that lacks important metadata. The next notebook will go through the steps of locating and adding relevant metadata to the backscatter data cubes read in this notebook.
